In [ ]:
import intake
from easygems import healpix as egh

import numpy as np
import xarray as xr
import healpy

import scipy.stats

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Time period

# time = ('2020-04-01','2020-04-30')
time = ('2020-08-01','2020-08-31')
month = 'August' # for plot names

# Region

domains10x10 = {
    "peruvian":     np.array([-90, -80, -20, -10]) ,
    "namibian":     np.array([0, 10, -20, -10]),
    "californian":  np.array([-130, -120, 20, 30]),
    "canarian":     np.array([-35, -25, 15, 25])
}

map_domain = domains10x10['namibian'] + np.array([-1,1,-1,1])*0 

In [ ]:
# Load dataset

hknode = 'EU'
sim = 'icon_d3hp003'
zoom = 11


cat = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")[hknode]

ds_globe = cat[sim](time_method='inst',time='PT6H',zoom=zoom).to_dask().pipe(egh.attach_coords)

In [ ]:
# Select time period and and region

cells = egh.isel_extent(ds_globe,map_domain+np.array([-1,1,-1,1])*0.1) # a little margin on each side is needed for a smooth remapping into lon-lat

ds = ds_globe.sel(time=slice(*time)).isel(cell=cells)

In [ ]:
%%time

# Derive relevant parameters


# Integrate qall
ds['qallvi'] = (ds['qall'].integrate('pressure')/9.81) \
    .assign_attrs(long_name='Total condensate', units='kg/m^2')


# Find max and min column velocity
ds['wamax'] = ds['wa'].sel(pressure=slice(900e2,1000e2)).max(dim='pressure') \
        .assign_attrs(long_name='Max vert velocity below 900 hPa', units='m/2')

ds['wamin'] = ds['wa'].sel(pressure=slice(900*100,1000*100)).min(dim='pressure') \
        .assign_attrs(long_name='Min vert velocity below 900 hPa', units='m/2')


# Change units for precip
ds['pr'].data = ds['pr'].data*3600*24/1e3*1e3
ds['pr'].attrs['units'] = 'mm/day'


# Calculate simple stats
ds['qallvi_mean'] = ds['qallvi'].mean(dim='cell').assign_attrs(long_name='Total condensate MEAN', units='kg/m^2')
ds['qallvi_std']  = ds['qallvi'].std(dim='cell').assign_attrs(long_name='Total condensate STD', units='kg/m^2')
ds['qallvi_skw']  = ds['qallvi'].reduce(scipy.stats.skew,dim='cell').assign_attrs(long_name='Total condensate SKW', units='')
ds['qallvi_hom']  = (ds['qallvi_mean']/ds['qallvi_std']).assign_attrs(long_name='Total condensate MEAN/STD', units='')

ds['pr_avg'] = ds['pr'].mean(dim='cell').assign_attrs(long_name='Precipitation MEAN', units='mm/day')

In [ ]:
# Remapping function from easygems

def get_nn_lon_lat_index(nside, lons, lats):
    lons2, lats2 = np.meshgrid(lons, lats)
    return xr.DataArray(
        healpy.ang2pix(nside, lons2, lats2, nest=True, lonlat=True),
        coords=[("lat", lats), ("lon", lons)],
    )

In [ ]:
%%time
# Remap to regular lon-lat

dlon = map_domain[1]-map_domain[0]
dlat = map_domain[3]-map_domain[2]

# Estimate number of needed grid points
nlon = np.ceil(dlon/360 * 4*2**zoom *np.sqrt(2)).astype(int)
nlat = np.ceil(nlon * dlat/dlon).astype(int)

# Approximate grid cell size in km
res = dlat*111/nlat

# Supersampling to avoid aliasing following easygems
supersampling = {"lon": 4, "lat": 4}

idx = get_nn_lon_lat_index(
    2**zoom,
    np.linspace(map_domain[0], map_domain[1], supersampling["lon"] * nlon),
    np.linspace(map_domain[2], map_domain[3], supersampling["lat"] * nlat),
)

remap_qallvi = ds['qallvi'].drop_vars(('lon','lat')).sel(cell=idx).coarsen(supersampling).mean()

In [ ]:
# Cloud mask based on threshold

remap_cloudmask = (remap_qallvi > 1e-1) \
    .assign_attrs(long_name='Cloud mask', units='')

In [ ]:
# Calculate LvL score

from tools.LvL import LvL

Nt = len(remap_cloudmask.time)
KS_cloud = np.zeros(Nt)
KS_void = np.zeros(Nt)
cd, cdt, vd, vdt = [None]*Nt,[None]*Nt,[None]*Nt,[None]*Nt

for i,t in enumerate(remap_cloudmask.time):
    KS_cloud[i], KS_void[i], cd[i], cdt[i], vd[i], vdt[i] = LvL(remap_cloudmask.sel(time=t).values)

In [ ]:
# Plot timeseries

plot_path = f"./figures/"

variables = ['qallvi_mean','qallvi_std','qallvi_skw','qallvi_hom','pr_avg']


Npanel = len(variables)+2
fig, axs = plt.subplots(Npanel,1, figsize=(12,3*Npanel), sharex=True, constrained_layout=True)

# Basic statistics
for ax, var in zip(axs,variables):
    ax.plot(ds.time, ds[var])
    ax.set_ylabel(f"{ds[var].attrs['long_name']:s} [{ds[var].attrs['units']:s}]")
    ax.set_xlim(ds.time[0],ds.time[-1])
    ax.grid()

# Cloud cover
ax = axs[-2]
ax.plot(remap_cloudmask.time, remap_cloudmask.mean(dim=('lat','lon')))
ax.set_ylabel('Cloud fraction')
ax.set_xlim(remap_cloudmask.time[0],remap_cloudmask.time[-1])
ax.grid()

# LvL scores
ax = axs[-1]
ax.plot(remap_cloudmask.time, KS_cloud, label='cloud chord lengths')
ax.plot(remap_cloudmask.time, KS_void, label='void chord lengths')
ax.set_ylabel('deviation from randomnes')
ax.grid()
ax.legend()
ax.set_xlim(remap_cloudmask.time[0],remap_cloudmask.time[-1])

plt.suptitle(sim+' in '+month)
plt.savefig(plot_path+'lvl_timeseries_'+sim+'_'+month,bbox_inches='tight',dpi=300)

In [ ]:
# Plot correlation KS_cloud vs KS_void

plot_path = f"./figures/"

gh = remap_cloudmask.groupby('time.hour').groups

plt.figure()
for h in [0,6,12,18]:
    plt.plot(KS_cloud[gh[h]],KS_void[gh[h]],'o',label=f"{h:02d}:00")
plt.grid()
plt.legend()
plt.xlabel('deviation from randomness: cloud chord lengths')
plt.ylabel('deviation from randomness: void chord lengths')
plt.title(sim+' in '+month)
plt.savefig(plot_path+'lvl_scatter_'+sim+'_'+month,bbox_inches='tight',dpi=300)

In [ ]:
# Collapse distributions

def sum_dist(cd):
    maxln = np.max([x.size for x in cd])
    CD = np.zeros((len(cd), maxln))
    for i,x in enumerate(cd):
        CD[i,0:x.size] = x
    return np.sum(CD,axis=0)


gh = remap_cloudmask.groupby('time.hour').groups

gh['all'] = np.arange(0,len(remap_cloudmask.time),1).tolist()


CD, VD, CDT, VDT, P = {},{},{},{},{}

for k in gh.keys():
    CD[k] = sum_dist([cd[ind] for ind in gh[k]])
    VD[k] = sum_dist([vd[ind] for ind in gh[k]])

    Npix = np.prod(remap_cloudmask.isel(time=gh[k]).shape)
    p = remap_cloudmask.isel(time=gh[k]).mean().values
    CDT[k] = (Npix * (1 - p) ** 2) * p ** np.arange(1, CD[k].size+1)
    q = 1 - p
    VDT[k] = (Npix * (1 - q) ** 2) * q ** np.arange(1, VD[k].size+1)
    P[k] = p

In [ ]:
# Plot collapsed distributions

Npanel = len(gh)

fig, axs = plt.subplots(1, Npanel, figsize=(4*Npanel,4), sharey=True, constrained_layout=True)

for ax, k in zip(axs,gh.keys()):

    lc = np.arange(1,len(CD[k])+1)*res
    lv = np.arange(1,len(VD[k])+1)*res
    ax.plot(lc,CD[k]/np.sum(CD[k]),label='cloud observed')
    ax.plot(lv,VD[k]/np.sum(VD[k]),label='void observed')
    ax.plot(lc,CDT[k]/np.sum(CDT[k]),'--',label='cloud random')
    ax.plot(lv,VDT[k]/np.sum(VDT[k]),'--',label='void random')
    ax.grid()
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylim(1e-6,1)
    ax.set_xlim(res,1e3)
    ax.set_title(f"T={str(k):s} CF={P[k]:.2f}")
    ax.set_xlabel('Chord length [km]')

axs[0].set_ylabel('Probability')
axs[0].legend()

plt.suptitle(sim+' in '+month)
plt.savefig(plot_path+'lvl_dist_'+sim+'_'+month,bbox_inches='tight',dpi=300)

In [ ]:
# Map plotting functions

def draw_map(ax,map_domain):
    ax.set_extent(map_domain, crs=ccrs.PlateCarree())
    ax.add_feature(cf.NaturalEarthFeature('physical', 'land', '10m'),
               facecolor='none', edgecolor='red', linewidth=1)
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

def plot_annotate(ax,da):
    cb = plt.colorbar(im, ax=ax, shrink=0.9, aspect=30, pad=0.02, extend='max',
                      label=f"{da.attrs['long_name']:s} [{da.attrs['units']:s}]")
    datestr = da.time.values.astype('datetime64[h]').item().strftime('%Y-%m-%dT%H')
    if 'healpix_nside' in da.crs.attrs:
        zoom = np.log2(da.crs.healpix_nside).astype(int)
        ax.set_title(datestr)
    else:
        ax.set_title(datestr)

In [ ]:
# Plot example maps and distributions

time_step = np.timedelta64(10,'D') 

plot_path = f"./figures/lvl_ex/"
# plot_path = f"/work/bb1153/b382422/global_hackathon/lvl_4x4/"
figsize = (14,4)

for t in np.arange(remap_qallvi.time[0].values,remap_qallvi.time[-1].values,time_step):

    fig, axs = plt.subplots(1,3,figsize=figsize,
                            subplot_kw={"projection": ccrs.PlateCarree()},
                            constrained_layout=True)
    
    # LWP in original healpix grid

    da = ds['qallvi'].sel(time=t)

    ax = axs[0]
    draw_map(ax,map_domain)
    im = egh.healpix_show(da, ax=ax, cmap='Blues_r',vmin=0,vmax=1)
    plot_annotate(ax,da)
    
    # LWP remapped to lon-lat

    # da = remap_qallvi.sel(time=t)
    
    # ax = axs[1]
    # draw_map(ax,map_domain)
    # im = ax.imshow(da, extent=ax.get_xlim() + ax.get_ylim(), origin="lower",
    #                cmap='Blues_r',vmin=0,vmax=1)
    # plot_annotate(ax,da)

    # Cloudmask in lon-lat

    da = remap_cloudmask.sel(time=t)
    
    ax = axs[1]
    draw_map(ax,map_domain)
    im = ax.imshow(da, extent=ax.get_xlim() + ax.get_ylim(), origin="lower",
                   cmap='Blues_r',vmin=0,vmax=1)
    plot_annotate(ax,da)

    # Cloud/void chord length distribution
    
    axs[-1].remove()
    ax = fig.add_subplot(1,3,3)

    i = np.where(remap_cloudmask.time.values==t)[0][0]
    lc = np.arange(1,len(cd[i])+1)*res
    lv = np.arange(1,len(vd[i])+1)*res
    ax.plot(lc,cd[i]/np.sum(cd[i]),label='cloud observed')
    ax.plot(lv,vd[i]/np.sum(vd[i]),label='void observed')
    ax.plot(lc,cdt[i]/np.sum(cdt[i]),'--',label='cloud random')
    ax.plot(lv,vdt[i]/np.sum(vdt[i]),'--',label='void random')
    ax.grid()
    ax.set_xlabel('Chord length [km]')
    ax.set_ylabel('Probability')
    ax.legend()
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylim(1e-3,1)
    ax.set_xlim(res,1e3)
    ax.set_title(f"dev. from random: cloud = {KS_cloud[i]:.2f}, void = {KS_void[i]:.2f}")

    # plt.suptitle(sim+' in '+month)
    datestr = t.astype('datetime64[h]').item().strftime('%Y-%m-%dT%H')
    plt.savefig(plot_path+datestr,bbox_inches='tight',dpi=300)

    print(datestr)
    # plt.close(fig)